In [1]:
#!pip install selenium
#!pip install chromedriver-binary-auto
!pip install webdriver-manager

In [2]:
import time

from selenium import webdriver
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

#options = Options()
#options.add_argument('--headless')    # ヘッドレスモードに
#driver = webdriver.Chrome(options=options)
driver = webdriver.Chrome(ChromeDriverManager().install())
wait = WebDriverWait(driver,10)

URL = "https://db.netkeiba.com/?pid=race_search_detail"
driver.get(URL)
time.sleep(1)
wait.until(EC.presence_of_all_elements_located)

# 月ごとに検索
year = 2023
month = 1

# 期間を選択
#start_year_element = driver.find_element_by_name('start_year')
start_year_element =driver.find_element(By.NAME,"start_year")
start_year_select = Select(start_year_element)
start_year_select.select_by_value(str(year))
#start_mon_element = driver.find_element_by_name('start_mon')
start_mon_element = driver.find_element(By.NAME,"start_mon")
start_mon_select = Select(start_mon_element)
start_mon_select.select_by_value(str(month))
#end_year_element = driver.find_element_by_name('end_year')
end_year_element = driver.find_element(By.NAME,"end_year")
end_year_select = Select(end_year_element)
end_year_select.select_by_value(str(year))
#end_mon_element = driver.find_element_by_name('end_mon')
end_mon_element = driver.find_element(By.NAME,"end_mon")
end_mon_select = Select(end_mon_element)
end_mon_select.select_by_value(str(month))

# 中央競馬場をチェック
for i in range(1,11):
    #terms = driver.find_element_by_id("check_Jyo_"+ str(i).zfill(2))
    terms = driver.find_element(By.ID,"check_Jyo_"+ str(i).zfill(2))
    terms.click()
        
# 表示件数を選択(20,50,100の中から最大の100へ)
#list_element = driver.find_element_by_name('list')
list_element = driver.find_element(By.NAME,"list")
list_select = Select(list_element)
list_select.select_by_value("100")

# フォームを送信
#frm = driver.find_element_by_css_selector("#db_search_detail_form > form")
frm = driver.find_element(By.CSS_SELECTOR,"#db_search_detail_form > form")
frm.submit()
time.sleep(5)
wait.until(EC.presence_of_all_elements_located)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:00<00:00, 73.7MB/s]
C:\Users\kawam\AppData\Local\Temp\ipykernel_1532\1027089030.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


<function selenium.webdriver.support.expected_conditions.presence_of_all_elements_located.<locals>._predicate(driver)>

In [3]:
with open(str(year)+"-"+str(month)+".txt", mode='w') as f:
    while True:
        time.sleep(5)
        wait.until(EC.presence_of_all_elements_located)
        #all_rows = driver.find_element_by_class_name('race_table_01').find_elements_by_tag_name("tr")
        all_rows = driver.find_element(By.CLASS_NAME,"race_table_01").find_elements(By.TAG_NAME,"tr")
        for row in range(1, len(all_rows)):
            #race_href=all_rows[row].find_elements_by_tag_name("td")[4].find_element_by_tag_name("a").get_attribute("href")
            race_href=all_rows[row].find_elements(By.TAG_NAME,"td")[4].find_element(By.TAG_NAME,"a").get_attribute("href")
            f.write(race_href+"\n")
        try:
            #target = driver.find_elements_by_link_text("次")[0]
            target = driver.find_elements(By.LINK_TEXT,"次")[0]
            driver.execute_script("arguments[0].click();", target) #javascriptでクリック処理
        except IndexError:
            break

In [4]:
import os
import requests

save_dir = "html"+"/"+str(year)+"/"+str(month)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
        
with open(str(year)+"-"+str(month)+".txt", "r") as f:
    urls = f.read().splitlines()
    for url in urls:
        list = url.split("/")
        race_id = list[-2]
        save_file_path = save_dir+"/"+race_id+'.html'
        response = requests.get(url)
        response.encoding = response.apparent_encoding
        html = response.text
        time.sleep(5)
        with open(save_file_path, 'w') as file:
            file.write(html)

In [5]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

CSV_DIR = "csv"
if not os.path.isdir(CSV_DIR):
    os.makedirs(CSV_DIR)
save_race_csv = CSV_DIR+"/race-"+str(year)+"-"+str(month)+".csv"
horse_race_csv = CSV_DIR+"/horse-"+str(year)+"-"+str(month)+".csv"

# race_data_columns, horse_data_columnsは長くなるので省略
race_df = pd.DataFrame(columns=race_data_columns )
horse_df = pd.DataFrame(columns=horse_data_columns )

html_dir = "html"+"/"+str(year)+"/"+str(month)
if os.path.isdir(html_dir):
    file_list = os.listdir(html_dir)
    for file_name in file_list:
        with open(html_dir+"/"+file_name, "r") as f:
            html = f.read()
            list = file_name.split(".")
            race_id = list[-2]
            race_list, horse_list_list = get_rade_and_horse_data_by_html(race_id, html) # 長くなるので省略
            for horse_list in horse_list_list:
                horse_se = pd.Series( horse_list, index=horse_df.columns)
                horse_df = horse_df.append(horse_se, ignore_index=True)
            race_se = pd.Series(race_list, index=race_df.columns )
            race_df = race_df.append(race_se, ignore_index=True )
            
race_df.to_csv(save_race_csv, header=True, index=False)
horse_df.to_csv(horse_race_csv, header=True, index=False)

NameError: name 'race_data_columns' is not defined

In [ ]:
# 時間情報を抜き出して、日付情報と結合。datetime型にする
race_df["time"] = race_df["time"].str.replace('発走 : (\d\d):(\d\d)(.|\n)*', r'\1時\2分')
race_df["date"] = race_df["date"] + race_df["time"]
race_df["date"] = pd.to_datetime(race_df['date'], format='%Y年%m月%d日%H時%M分')
# もともとのtimeは不要なので削除
race_df.drop(['time'], axis=1, inplace=True)

# 何ラウンド目かのカラムに余分なRや空白・改行が含まれているので取り除く
race_df['race_round'] = race_df['race_round'].str.strip('R \n')